In [10]:
#last run 02/05/2024 0541
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import transforms, datasets
from torchsummary import summary
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [11]:
import os

In [12]:
files = os.path.join("your file name")

In [13]:
#Defining the transformations to apply on our dataset
transformations = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

In [14]:
# Load your dataset
dataset = datasets.ImageFolder(root='your file name', transform=transformations)

In [15]:
# Define the sizes of your splits
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

In [16]:
# Randomly split the dataset into train and test
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [17]:
# Create DataLoaders for your train and test datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [18]:
import import_ipynb
from adamsop_DenseNet import DenseNetModel


In [19]:
import import_ipynb
from adamsop import ThreeHandsArchitecture

In [ ]:
# Instantiate models
three_hands_model = ThreeHandsArchitecture()
densenet_model = DenseNetModel(num_classes=7)

# Load pre-trained state dictionaries
three_hands_model.load_state_dict(torch.load('/Users/atishayjain/Downloads/three_hands_model.pth'))
densenet_model.load_state_dict(torch.load('/Users/atishayjain/Downloads/Densenet_model.pth'))

# Set models to evaluation mode if you are using them for inference
three_hands_model.eval()
densenet_model.eval()

In [21]:
class CombinedModel(nn.Module):
    def __init__(self, three_hands, densenet, num_classes=7):
        super(CombinedModel, self).__init__()
        self.three_hands = three_hands
        self.densenet = densenet
        self.fc = nn.Linear(num_classes * 2, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Assuming both models output probabilities that can be concatenated
        output_three_hands = self.three_hands(x)
        output_densenet = self.densenet(x)

        concatenated_output = torch.cat((output_three_hands, output_densenet), dim=1)
        output = self.fc(concatenated_output)
        output = self.softmax(output)
        return output

# Instantiate the combined model with pre-trained models
combined_model = CombinedModel(three_hands_model, densenet_model)


In [32]:
#Defing the hyperparameters
num_epochs = 3
learning_rate = 0.005

In [33]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(combined_model.parameters(), lr=learning_rate,weight_decay=0.0001)

In [ ]:
# Training loop
loss_values = []
for epoch in range(num_epochs):
    combined_model.train()
    running_loss = 0.0
    loop = tqdm(train_loader, leave=True, desc=f"Epoch {epoch+1}/{num_epochs}")
    iterations = 0
    acc_loss = 0
    for inputs, labels in loop:
        inputs, labels = inputs, labels
        optimizer.zero_grad()
        outputs = combined_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        iterations += 1
        acc_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    loss_value = acc_loss/iterations
    loss_values.append(loss_value)
    print(f'\nEpoch [{epoch+1}/{num_epochs}], Loss: {loss_value:.4f}')

In [ ]:
# Testing our model
combined_model.eval()
correct = 0
total = 0

with torch.no_grad():
    loop = tqdm(train_loader, leave=True, desc=f"Epoch {epoch+1}/{num_epochs}")
    # for inputs, labels in test_loader:
    for inputs, labels in loop:
        inputs, labels = inputs, labels
        outputs = combined_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loop.set_postfix()
accuracy = correct / total
print(f'\nTrain Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Testing our model
combined_model.eval()
correct = 0
total = 0

with torch.no_grad():
    loop = tqdm(test_loader, leave=True, desc=f"Epoch {epoch+1}/{num_epochs}")
    # for inputs, labels in test_loader:
    for inputs, labels in loop:
        inputs, labels = inputs, labels
        outputs = combined_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loop.set_postfix()
accuracy = correct / total

print(f'\nTest Accuracy: {accuracy * 100:.2f}%')